In [1]:
import numpy as np
import os
import tensorflow as tf
import keras
import optuna

BATCH_SIZE = 4096
DATA_TFREC = "data"
# create list of all tfrec files in folder and sub folders
TFREC_FILES = tf.io.gfile.glob(DATA_TFREC + '/**/*.tfrecord')
print(len(TFREC_FILES))
train_files = TFREC_FILES[1:46]
valid_files = TFREC_FILES[46:48]
train_options = tf.data.Options()
train_options.deterministic = True

def _parse_function(example_proto):
    feature_description = {
        'x': tf.io.FixedLenFeature([556], tf.float32),
        'targets': tf.io.FixedLenFeature([368], tf.float32)
    }
    e = tf.io.parse_single_example(example_proto, feature_description)
    return e['x'], e['targets'][60:120]

ds_train = (
  tf.data.TFRecordDataset(train_files, compression_type="GZIP")
    .with_options(train_options)
    .shuffle(100)
  .map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(4 * BATCH_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

norm_x = keras.layers.Normalization()
norm_x.adapt(ds_train.map(lambda x, y: x).take(1000))
mean_x = norm_x.mean
stdd_x = keras.ops.maximum(1e-10, norm_x.variance ** 0.5)

norm_y = keras.layers.Normalization()
norm_y.adapt(ds_train.map(lambda x, y: y).take(1000))
mean_y = norm_y.mean
stdd_y =keras.ops.maximum(1e-10, norm_y.variance ** 0.5)

del ds_train

/Users/johnny/Library/CloudStorage/OneDrive-Personal/py/Kaggle/LEAP24/leap24/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


47


2024-06-21 22:31:04.502124: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-21 22:32:50.167274: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
from kan import KAN
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
import numpy as np

class NumpyDataset(Dataset):
    def __init__(self, folder_path, mean_x, stdd_x, mean_y, stdd_y, train=True):
        self.files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.npy')][:5]
        self.train = train
        if train:
            self.files = self.files[:int(len(self.files) * 0.8)]
        else:
            self.files = self.files[int(len(self.files) * 0.8):]
        
        self.mean_x = mean_x
        self.stdd_x = stdd_x
        self.mean_y = mean_y
        self.stdd_y = stdd_y

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file_path = self.files[idx]
        data = np.load(file_path)
        features = data[:,:556]
        targets = data[:,556+60:556+120]

        # Normalize features and targets
        features = (features - self.mean_x) / self.stdd_x
        targets = (targets - self.mean_y) / self.stdd_y
        
        return torch.tensor(features, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

mean_x = np.array(mean_x)
stdd_x = np.array(stdd_x)
mean_y = np.array(mean_y)
stdd_y = np.array(stdd_y) 

# Usage
folder_path = 'data'
train_set = NumpyDataset(folder_path, mean_x, stdd_x, mean_y, stdd_y, train=True)
val_set = NumpyDataset(folder_path, mean_x, stdd_x, mean_y, stdd_y, train=False)
trainloader = DataLoader(train_set, batch_size=1, shuffle=True)
valloader = DataLoader(val_set, batch_size=1, shuffle=False)

def clipped_r2_score(y_true, y_pred):
    ss_res = torch.sum((y_true - y_pred) ** 2)
    ss_tot = torch.sum((y_true - torch.mean(y_true)) ** 2)
    r2 = 1 - ss_res / ss_tot
    clipped_r2 = torch.clamp(r2, 0.0, 1.0)
    return clipped_r2

# Define the hyperparameter tuning function
def objective(trial):
    # Suggest hyperparameters
    layer2 = trial.suggest_int('layer2', 30, 500)
    layer3 = trial.suggest_int('layer3', 30, 300)
    spline_order = trial.suggest_categorical('spline_order', [1, 2, 3, 4, 5])
    grid_size = trial.suggest_int('grid_size', 10, 100)
    grid_range1 = trial.suggest_categorical('grid_range1', [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0])
    grid_range2 = trial.suggest_categorical('grid_range2', [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0])
    
    # Define model with the suggested hyperparameters
    model = KAN([556, layer2, layer3, 60],
                grid_size=grid_size,
                spline_order=spline_order,
                grid_range=[grid_range1, grid_range2])
    device = torch.device("cpu")
    model.to(device)
    
    # Define optimizer
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=12, eta_min=5e-6, last_epoch=-1)
    
    # Define loss
    criterion = nn.MSELoss()
    
    # Training loop
    for epoch in range(12):
        # Train
        model.train()
        with tqdm(trainloader) as pbar:
            for i, (features, targets) in enumerate(pbar):
                features = features.to(device)
                targets = targets.to(device)
                
                optimizer.zero_grad()
                output = model(features)
                loss = criterion(output, targets)
                loss.backward()
                optimizer.step()
                
                r2_score = clipped_r2_score(targets, output)
                pbar.set_postfix(loss=loss.item(), r2_score=r2_score.item(), lr=optimizer.param_groups[0]['lr'])
        
        # Validation
        model.eval()
        val_loss = 0
        val_r2_score = 0
        with torch.no_grad():
            for features, targets in valloader:
                features = features.to(device)
                targets = targets.to(device)
                output = model(features)
                val_loss += criterion(output, targets).item()
                val_r2_score += clipped_r2_score(targets, output).item()
        
        val_loss /= len(valloader)
        val_r2_score /= len(valloader)
        
        # Update learning rate
        scheduler.step()
    
    # Return the validation R2 score as the metric to optimize
    return val_r2_score

# Create a study and optimize it
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

# Print the best trial
print(f"Best trial: {study.best_trial}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-06-21 22:32:51,245] A new study created in memory with name: no-name-9c2e8c69-2850-4f57-8661-342258810256
100%|██████████| 4/4 [00:27<00:00,  6.92s/it, loss=0.6, lr=2.2e-5, r2_score=0.186]  
[I 2024-06-21 22:39:32,238] Trial 0 finished with value: 0.1478651762008667 and parameters: {'layer2': 86, 'layer3': 32, 'spline_order': 1, 'grid_size': 38, 'grid_range1': 2.0, 'grid_range2': 1.0}. Best is trial 0 with value: 0.1478651762008667.
100%|██████████| 4/4 [01:52<00:00, 28.06s/it, loss=0.525, lr=2.2e-5, r2_score=0.208]
[I 2024-06-21 23:05:04,611] Trial 1 finished with value: 0.20468688011169434 and parameters: {'layer2': 219, 'layer3': 265, 'spline_order': 3, 'grid_size': 25, 'grid_range1': -3.0, 'grid_range2': 3.0}. Best is trial 1 with value: 0.20468688011169434.
100%|██████████| 4/4 [05:09<00:00, 77.34s/it, loss=0.563, lr=2.2e-5, r2_score=0.236]
[I 2024-06-22 00:15:36,540] Trial 2 finished with value: 0.18696719408035278 and parameters: {'layer2': 128, 'layer3': 120, 'spline_or